<a href="https://colab.research.google.com/github/Sairohit022001/RAG-POWERED-EVENT-BOT/blob/main/RAG_POWERED_EVENT_ASSISTANT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install streamlit pyngrok --quiet

In [28]:
%%writefile app.py
import streamlit as st
from agenda import show_agenda
from matching import show_matching
from directions import show_directions
from recommendations import show_recommendations
from lunch_timer import show_lunch_timer
from feedback import show_feedback
from rag_agent import show_rag_interface

# Page config
st.set_page_config(page_title="Event Bot", page_icon="⚙︎")

# Sidebar Navigation
option = st.sidebar.selectbox("Choose an option", [
    "Agenda",
    "Matching",
    "Recommendations",
    "Directions",
    "Lunch Timer",
    "Event Feedback",
    "Event Assistant"
])

# Only show welcome message on Agenda page
if option == "Agenda":
    st.title("⚙︎ WELCOME TO RAG POWERED EVENT ASSISTANT")
    st.markdown("""
    This is your **Build with AI Workshop Event Bot** — Your Smart Assistant for Event Info, Schedule & Updates
    1. ✅ Agenda Details
    2. 👥 Participant Matching Engine (Resume-Based)
    3. 📌 Session recommendations
    4. 🗺 Washroom directions
    5. ⏱ Real-time lunch countdown
    6. 📝 Session Feedback (Conversational)
    7. 🤖 Event Assistant (Chat with your Event)
    """)
    show_agenda()

elif option == "Matching":
    show_matching()
elif option == "Recommendations":
    show_recommendations()
elif option == "Directions":
    show_directions()
elif option == "Lunch Timer":
    show_lunch_timer()
elif option == "Event Feedback":
    show_feedback()
elif option == "Event Assistant":
    show_rag_interface()


Overwriting app.py


In [29]:

import streamlit as st
from sentence_transformers import SentenceTransformer , util

model = SentenceTransformer("all-MiniLM-L6-v2")

In [30]:
%%writefile agenda.py

import streamlit as st

def show_agenda():


    agenda = {
       "10:00 - 11:00 AM": "Hands On Workshop: Automation using Claude and MCP Server - Vijender P, Alumnx",
       "11:00 - 12:00 PM": "Hands On Workshop: Agentic AI - Jitendra Gupta (Google Developer Expert)",
       "12:00 - 1:00 PM": "Industry Connect Session - Ravi Babu, Apex Cura Healthcare",
       "1:00 - 2:00 PM": "Lunch",
       "2:00 - 3:00 PM": "Hands On Workshop: Build an Event Bot using RAG - Vishvas Dubey, TCS",
       "3:00 - 3:30 PM": "Industry Application of AI: Building Multi AI Agents - Surendranath Reddy, QAPilot",
       "3:30 - 4:00 PM": "Workshop: Building Multi AI Agents - Mahidhar, NexusHub"
    }

    st.header(" Meeting Agenda")
    for time, event in agenda.items():
        st.markdown(f"**{time}**: {event}")


Overwriting agenda.py


In [31]:
%%writefile matching.py

import streamlit as st
import pdfplumber
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

if "resumes" not in st.session_state:
    st.session_state.resumes = {}

def extract_text_from_pdf(file):
    with pdfplumber.open(file) as pdf:
        text = ""
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

def show_matching():
    st.sidebar.header("👥 Participant Matching Engine (Resume-Based)")

    uploaded_file = st.file_uploader("Upload your resume (PDF only)", type=["pdf"])
    if uploaded_file is not None:
        resume_text = extract_text_from_pdf(uploaded_file)
        user_id = f"user_{len(st.session_state.resumes)+1}"
        st.session_state.resumes[user_id] = resume_text
        st.success("Resume uploaded and processed!")

        st.subheader("Your Resume Preview")
        st.text_area("Resume Content", resume_text, height=200)

    if len(st.session_state.resumes) > 1:
        st.subheader("Matching Participants")
        texts = list(st.session_state.resumes.values())
        user_ids = list(st.session_state.resumes.keys())

        vectorizer = TfidfVectorizer(stop_words="english")
        tfidf_matrix = vectorizer.fit_transform(texts)

        similarity_matrix = cosine_similarity(tfidf_matrix)

        for idx, user_id in enumerate(user_ids):
            st.write(f"Matches for **{user_id}**:")
            sim_scores = similarity_matrix[idx]
            sim_scores[idx] = -1
            top_matches = sim_scores.argsort()[-2:][::-1]

            for match_idx in top_matches:
                st.write(f"- {user_ids[match_idx]} (Similarity: {sim_scores[match_idx]:.2f})")
            st.write("---")
    else:
        st.info("Upload at least 2 resumes to see matching results.")


Overwriting matching.py


In [32]:
%%writefile directions.py

import streamlit as st

def show_directions():
    st.header("🗺 Washroom Directions")

    st.markdown("""
     **Venue Layout**:
    -  Ground Floor:
        - 🚻 Washroom near Registration Desk
        - 🚺 Women’s Washroom next to Conference Room A
    -  First Floor:
        - 🚻 Common Washroom near Workshop Hall
        - ♿ Accessible Washroom at the stairway entrance

    🔄 Ask any volunteer if you feel lost – they’re happy to guide you!
    """)


Overwriting directions.py


In [33]:
%%writefile recommendations.py
import streamlit as st

def show_recommendations():
    st.header("📌 Session Recommendations")
    st.write("Based on your interests, here are some sessions we think you’ll love:")
    recommendations = [
        "🤖 Hands On Workshop: Automation using Claude and MCP Server",
        "🧠 Agentic AI - Jitendra Gupta (Google Developer Expert)",
        "🤝 Industry Application of AI: Building Multi AI Agents - Surendranath Reddy",
        "💡 Workshop: Building Multi AI Agents - Mahidhar, NexusHub"
    ]
    for session in recommendations:
        st.markdown(f"- {session}")


Overwriting recommendations.py


In [34]:
%%writefile lunch_timer.py
import streamlit as st
import time
from datetime import datetime, timedelta


def show_lunch_timer():
    st.header("⏱️ Real-time Lunch Countdown")

    lunch_end_time = datetime(year=2025, month=5, day=18, hour=13, minute=0, second=0)


    if datetime.now() > lunch_end_time:
        st.success("🍽️ Lunch time is over!")
        return

    placeholder = st.empty()

    while True:
        now = datetime.now()
        remaining = lunch_end_time - now

        if remaining.total_seconds() <= 0:
            placeholder.success("🍽️ Lunch time is over!")
            break

        mins, secs = divmod(int(remaining.total_seconds()), 60)
        placeholder.markdown(f"Time remaining for lunch: **{mins} minutes {secs} seconds**")

        time.sleep(1)

Overwriting lunch_timer.py


In [35]:
%%writefile feedback.py

import streamlit as st

feedback_list = []

def show_feedback():
    st.header("📝 Session Feedback")

    name = st.text_input("Your Name")
    session = st.selectbox("Session", ["Workshop 1", "Workshop 2", "Industry Talk", "Lunch"])
    rating = st.slider("Rate this session (1-5)", 1, 5, 3)
    comments = st.text_area("Additional Comments")

    if st.button("Submit Feedback"):
        if not name or not comments:
            st.warning("Please fill in your name and comments.")
        else:
            feedback = {
                "name": name,
                "session": session,
                "rating": rating,
                "comments": comments,
            }
            feedback_list.append(feedback)
            st.success("Thank you for your feedback!")

    if feedback_list:
        st.subheader("Previous Feedback")
        for fb in feedback_list:
            st.markdown(f"**{fb['name']}** on *{fb['session']}* rated: {fb['rating']}/5")
            st.write(fb['comments'])
            st.write("---")

Overwriting feedback.py


In [36]:

!pip install streamlit faiss-cpu sentence-transformers transformers


In [37]:
!pip install pdfplumber


In [42]:
%%writefile rag_agent.py

import streamlit as st
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

import numpy as np


embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
generator = pipeline('text2text-generation', model='t5-small')

embedding_dim = 384
index = faiss.IndexFlatL2(embedding_dim)
documents = []

def extract_text_from_pdf(file):
    with pdfplumber.open(file) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def chunk_text(text, max_length=500):
    sentences = text.split('. ')
    chunks = []
    chunk = ""
    for sent in sentences:
        if len(chunk) + len(sent) < max_length:
            chunk += sent + '. '
        else:
            chunks.append(chunk.strip())
            chunk = sent + '. '
    if chunk:
        chunks.append(chunk.strip())
    return chunks

def add_document(file):
    text = extract_text_from_pdf(file)
    chunks = chunk_text(text)
    embeddings = embedding_model.encode(chunks)
    for emb, chunk in zip(embeddings, chunks):
        index.add(np.array([emb]))
        documents.append(chunk)

def retrieve(query, top_k=3):
    query_emb = embedding_model.encode([query])
    D, I = index.search(np.array(query_emb), top_k)
    results = [documents[i] for i in I[0] if i < len(documents)]
    return results

def generate_answer(retrieved_chunks, query):
    context = " ".join(retrieved_chunks)
    input_text = f"summarize: {context} question: {query}"
    output = generator(input_text, max_length=150)[0]['generated_text']
    return output

def show_rag_interface():
    st.sidebar.header("📚 RAG Powered Event Assistant")

    uploaded_file = st.file_uploader("Upload", type=['pdf'])
    if uploaded_file is not None:
        add_document(uploaded_file)
        st.success("Document uploaded and indexed!")

    query = st.text_input("Ask anything:")

    if st.button("Get Answer") and query:
        if len(documents) == 0:
            st.warning("Please upload notes first!")
        else:
            retrieved = retrieve(query)
            answer = generate_answer(retrieved, query)
            st.markdown("**Answer:**")
            st.write(answer)


Overwriting rag_agent.py


In [39]:

!ngrok authtoken 2xAMlsU2bxIRl7KmMZbHhz7gtnA_2gRj1UMqL13VFrSrKzbaT


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:

from pyngrok import ngrok
import os

# Kill previous tunnels
!pkill streamlit

# Open tunnel
public_url = ngrok.connect(addr="8501", proto="http")

print(f"🔗 Streamlit app URL: {public_url}")

# Run Streamlit app
!streamlit run app.py &


🔗 Streamlit app URL: NgrokTunnel: "https://18cf-35-227-117-103.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.117.103:8501

2025-05-16 15:24:57.905528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747409097.937466   24865 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747409097.947135   24865 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Device set to use cpu
2025-05-16 15:25:06.462 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python

In [ ]:
pip freeze > requirements.txt
